In [1]:
# -*- coding: utf-8 -*-
import os
import codecs
def readfile(filename):
	#current_path=os.path.abspath(os.curdir)
	file_path=os.path.join("/content/drive/My Drive/SML_P1",filename)
	if not os.path.exists(file_path):
		print("error:file not found:"+filename)
		return ""
	f=codecs.open(file_path,"r","utf-8")
	s=f.read()
	f.close()
	return s
test_data=readfile("test-public.txt")
test_data=test_data.splitlines()
print("length of testing data:"+str(len(test_data)))
train_data=readfile("train.txt")
train_data=train_data.splitlines()
print("length of training data:"+str(len(train_data)))

length of testing data:2001
length of training data:20000


# Graph

## Train data

In [2]:
import numpy as np
positive_train = np.load("/content/drive/My Drive/SML_P1/positive_train_V2.npy").tolist()
negative_train = np.load("/content/drive/My Drive/SML_P1/negative_train_V2.npy").tolist()

In [3]:
print(positive_train[0:10])

[[1277201, 1098760], [1557862, 1209171], [362256, 4693295], [887693, 2381188], [2917576, 1847640], [2245586, 1881447], [11193, 1242964], [2064376, 3114820], [291591, 610015], [1336304, 1032341]]


## Test data

In [4]:
del test_data[0]

test_edges = []
for temp_str in test_data:
  temp = temp_str.split('\t')
  temp_list=[]
  temp_list.append(int(temp[1]))
  temp_list.append(int(temp[2]))
  test_edges.append(temp_list)

In [5]:
print(test_edges[0:10])

[[3563811, 3600160], [2052043, 1401960], [4517994, 1690636], [1660006, 4349447], [581111, 1882617], [1039683, 1365102], [4017576, 1291219], [1859935, 1059674], [4337137, 3648097], [2682050, 3501635]]


## Get undirected graph

In [ ]:
## unsuccessful
import networkx as nx

edge_list = []
import networkx as nx
G = nx.Graph()
for temp_str in train_data:
  temp=temp_str.split('\t')
  for i in range(1, len(temp)):
    temp_edge = (int(temp[0]), int(temp[i]))
    edge_list.append(temp_edge)
G.add_edges_from(edge_list)


In [7]:
##successful
import networkx as nx

G=nx.Graph()
for temp_str in train_data:
  temp=temp_str.split('\t')
  temp_list=[int(i) for i in temp]
  temp_edges=[(temp_list[0], temp_list[i]) for i in range(1, len(temp_list))]
  G.add_edges_from(temp_edges)

In [8]:
print('number of nodes',G.number_of_nodes())
print('number of edges',G.number_of_edges())

number of nodes 4867136
number of edges 23416061


In [11]:
## Get undirected_graph
edge_list = []
import networkx as nx
SG = nx.Graph()
for temp_str in train_data[0:2]:
  temp=temp_str.split('\t')
  for i in range(1, len(temp)):
    temp_edge = (int(temp[0]), int(temp[i]))
    edge_list.append(temp_edge)
SG.add_edges_from(edge_list)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(SG)

In [ ]:
print("Generating community......")
from networkx.algorithms import community

comms = list(community.asyn_fluidc(G,100))
print("Size of communities:"+str(len(comms)))

print("Adding community attribute......")
count=0
for node in G.nodes():
    if(count%100==0):
        print(count)
    count+=1
    for i in range(len(comms)):
        if node in comms[i]:
            G.nodes[node]['community'] = i

## features

In [25]:
def get_positive_features():
    features = []
    count = 0
    print("Generating positive features......")
    for temp_data in positive_train:
      if (count % 100 == 0):
        print(count)
      count += 1
      feature = []
      try:
        preds = nx.resource_allocation_index(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
        
        preds = nx.jaccard_coefficient(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
          
        # preds = nx.adamic_adar_index(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.nx.preferential_attachment(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.cn_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.ra_index_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)
        
        # preds = nx.within_inter_cluster(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)          
        
        feature.append(1)  # label=1
        
      except:
        print("one error at: "+str(count))
        pass
      
      features.append(feature)
    print("positive features: "+str(len(features)))
    return features

In [24]:
def get_negative_features():
    features = []
    count = 0
    print("Generating negative features......")
    for temp_data in negative_train:
      if (count % 100 == 0):
        print(count)
      count += 1
      feature = []
      try:
        preds = nx.resource_allocation_index(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
        
        preds = nx.jaccard_coefficient(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
          
        # preds = nx.adamic_adar_index(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.nx.preferential_attachment(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.cn_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.ra_index_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)
        
        # preds = nx.within_inter_cluster(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)          
        
        feature.append(0)  # label=1
        
      except:
        print("one error at: "+str(count))
        pass
      
      features.append(feature)
    print("positive features: "+str(len(features)))
    return features

In [23]:
def get_test_features():
    features = []
    count = 0
    print("Generating test data features......")
    for temp_data in test_edges:
      if (count % 100 == 0):
        print(count)
      count += 1
      feature = []
      try:
        preds = nx.resource_allocation_index(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
        
        preds = nx.jaccard_coefficient(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
          
        # preds = nx.adamic_adar_index(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.nx.preferential_attachment(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.cn_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)

        # preds = nx.ra_index_soundarajan_hopcroft(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)
        
        # preds = nx.within_inter_cluster(G, [temp_data])
        # for u, v, p in preds:
        #   feature.append(p)          
        
        
      except:
        print("one error at: "+str(count))
        pass
      
      features.append(feature)
    print("positive features: "+str(len(features)))
    return features

## Combine

In [26]:
#a = get_features(positive_train)
train_features = get_positive_features() + get_negative_features()

Generating positive features......
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
positive features: 10000
Generating negative features......
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500

In [27]:
print(train_features[0:10])
print(train_features[10000:10010])

[[0.0013062370994879372, 0.00641025641025641, 1], [0.037037037037037035, 0.0070921985815602835, 1], [0.021826836316432577, 0.02480916030534351, 1], [0.0023037541023116087, 0.006, 1], [0.03415841584158416, 0.011976047904191617, 1], [0.019494392024212215, 0.03773584905660377, 1], [0.36784845511546055, 0.021511017838405037, 1], [0, 0.0, 1], [0.0007380380154926836, 0.00881057268722467, 1], [0, 0.0, 1]]
[[0, 0.0, 0], [0, 0.0, 0], [0, 0.0, 0], [0, 0.0, 0], [2.496984890744426e-06, 0.00411522633744856, 0], [0, 0.0, 0], [1.6821706730364864e-05, 0.0005733944954128441, 0], [1.3112054304884109e-06, 0.005813953488372093, 0], [0, 0.0, 0], [0, 0.0, 0]]


In [31]:
test_features = get_test_features()

Generating test data features......
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
positive features: 2000


In [32]:
print(test_features[0:10])

[[0, 0.0], [0, 0.0], [0.0046237897060301255, 0.011152416356877323], [2.809273297149788e-05, 0.003669724770642202], [0, 0.0], [0, 0.0], [1.1639411045801083e-05, 0.0025188916876574307], [2.32067020955652e-05, 0.0019342359767891683], [0, 0.0], [0, 0.0]]


## Save features

In [33]:
import csv

with open("train_data.csv","w",newline="") as csvfile:
  writer=csv.writer(csvfile)
  writer.writerow(["RA","JC","Label"])
  #writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC","Label"])
  writer.writerows(train_features)

In [36]:
with open("test_data.csv","w",newline="") as csvfile:
  writer=csv.writer(csvfile)
  writer.writerow(["RA","JC"])
  #writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC"])
  writer.writerows(test_features)

In [35]:
!pwd

/content
